In [ ]:
from baynes.model_utils import *
from baynes.plotter import FitPlotter
from cmdstanpy import CmdStanModel
from baynes.analysis import standard_analysis
import matplotlib.pyplot as plt
import cmdstanpy
import logging
cmdstanpy.utils.get_logger().setLevel(logging.ERROR)
plt.style.use('ggplot')

In [ ]:
stan_file= get_stan_file("lorentzians_asymm.stan")
model = CmdStanModel(stan_file=stan_file,
                     **get_compiler_kwargs())

In [ ]:
datContent = [i.strip().split() for i in open("Fe_cal_7.dat").readlines()]
data = [float(d) for d in datContent[0]]
counts, edges = np.histogram(data, 100)
centers = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
plt.plot(centers, counts)

In [ ]:
data={"N_bins": len(counts), 
      'N_peaks': 2,
      'counts': counts.tolist(), 
      'x': edges,
      'N_ev': sum(counts), 
      'p_FWHM': 5, 
      'p_gamma': [3,0.8], 
      'p_E0': [5890, 5900],
      'p_i': [0.4,0.4],
      'prior': 1}

sampler_kwargs={
    'chains': 8,
    'iter_warmup': 500,
    'iter_sampling': 500,
    'save_warmup': True, 
}
p = FitPlotter(col_wrap=4)
fit = standard_analysis(model, data, p, sampler_kwargs=sampler_kwargs)

In [ ]:
fit.summary().loc['FWHM']

In [ ]:
stan_file= get_stan_file("poisGP.stan")
model = CmdStanModel(stan_file=stan_file,
                     **get_compiler_kwargs())

In [ ]:
data = {'N': len(counts),
        'x': centers,
        'y': counts}

sampler_kwargs={
    'chains': 16,
    'iter_warmup': 300,
    'iter_sampling': 300,
    'save_warmup': True, 
    'adapt_delta': 0.9,
    'max_treedepth': 12,
}
plot_pars=['rho', 'alpha', 'a', 'eta[10]']
fit = standard_analysis(model, data, p, sampler_kwargs, fit_title='m=0', data_key='y', rep_key='y_rep',plot_params=plot_pars)